In [ ]:
## Creamos la instancia de spark y agregamos el bucket S3
 # Como punto parametro para la creación del dataframe

from pyspark.sql import functions as F

In [ ]:
inputPath = 's3://aws-logs-035457841237-us-east-1/proyecto3/data.csv'
outputPath = 's3://aws-logs-035457841237-us-east-1/proyecto3/output/col_conf_dep'

In [ ]:
df = spark.read.format("csv").option("header", "true").load(inputPath)

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
new_names = list(map(lambda x: x.replace(" ", "_"), df.columns))
df = df.toDF(*new_names)

In [ ]:
df.printSchema()

In [ ]:
df.filter(df.ID_de_caso == 1).collect()

In [ ]:
mappedDf = df.select([c for c in df.columns if c in {'Departamento_o_Distrito_'}])
mappedDf.printSchema()

In [ ]:
mappedDf.show()

In [ ]:
mappedDf.filter(df.Departamento_o_Distrito_ == "Antioquia").collect()

In [ ]:
mappedDf.filter(df.Departamento_o_Distrito_ == "Amazonas").count()


In [ ]:
reducedDf = mappedDf.groupBy(mappedDf.Departamento_o_Distrito_).count()\
            .select(mappedDf.Departamento_o_Distrito_, F.col('count').alias('Confirmados')).orderBy(mappedDf.Departamento_o_Distrito_)
reducedDf.show(100)

In [ ]:
reducedDf.write.format("csv").mode("overwrite").option("header","true").save(outputPath)